<a href="https://colab.research.google.com/github/Pakapakk/Network_Traffic_Anomaly_Detection/blob/main/anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [ ]:
%%capture
!kaggle datasets download -d galaxyh/kdd-cup-1999-data
! unzip "kdd-cup-1999-data.zip"

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [123]:
kdd_df = pd.read_csv('kddcup.data_10_percent_corrected')

In [ ]:
kdd_df

In [11]:
with open("./kddcup.names",'r') as f:
    print(f.read())

back,buffer_overflow,ftp_write,guess_passwd,imap,ipsweep,land,loadmodule,multihop,neptune,nmap,normal,perl,phf,pod,portsweep,rootkit,satan,smurf,spy,teardrop,warezclient,warezmaster.
duration: continuous.
protocol_type: symbolic.
service: symbolic.
flag: symbolic.
src_bytes: continuous.
dst_bytes: continuous.
land: symbolic.
wrong_fragment: continuous.
urgent: continuous.
hot: continuous.
num_failed_logins: continuous.
logged_in: symbolic.
num_compromised: continuous.
root_shell: continuous.
su_attempted: continuous.
num_root: continuous.
num_file_creations: continuous.
num_shells: continuous.
num_access_files: continuous.
num_outbound_cmds: continuous.
is_host_login: symbolic.
is_guest_login: symbolic.
count: continuous.
srv_count: continuous.
serror_rate: continuous.
srv_serror_rate: continuous.
rerror_rate: continuous.
srv_rerror_rate: continuous.
same_srv_rate: continuous.
diff_srv_rate: continuous.
srv_diff_host_rate: continuous.
dst_host_count: continuous.
dst_host_srv_count: con

In [ ]:
with open("./training_attack_types",'r') as f:
    print(f.read())

back dos
buffer_overflow u2r
ftp_write r2l
guess_passwd r2l
imap r2l
ipsweep probe
land dos
loadmodule u2r
multihop r2l
neptune dos
nmap probe
perl u2r
phf r2l
pod dos
portsweep probe
rootkit u2r
satan probe
smurf dos
spy r2l
teardrop dos
warezclient r2l
warezmaster r2l




# Data Preprocessing

In [96]:
cols = """
duration,
protocol_type,
service,
flag,
src_bytes,
dst_bytes,
land,
wrong_fragment,
urgent,
hot,
num_failed_logins,
logged_in,
num_compromised,
root_shell,
su_attempted,
num_root,
num_file_creations,
num_shells,
num_access_files,
num_outbound_cmds,
is_host_login,
is_guest_login,
count,
srv_count,
serror_rate,
srv_serror_rate,
rerror_rate,
srv_rerror_rate,
same_srv_rate,
diff_srv_rate,
srv_diff_host_rate,
dst_host_count,
dst_host_srv_count,
dst_host_same_srv_rate,
dst_host_diff_srv_rate,
dst_host_same_src_port_rate,
dst_host_srv_diff_host_rate,
dst_host_serror_rate,
dst_host_srv_serror_rate,
dst_host_rerror_rate,
dst_host_srv_rerror_rate"""

In [97]:
# new_cols = [col.strip() for col in cols.split(',')]
# new_cols.append('target')
# kdd_df.columns = new_cols

In [55]:
kdd_df.isnull().sum()

,0
duration,0
protocol_type,0
service,0
flag,0
src_bytes,0
dst_bytes,0
land,0
wrong_fragment,0
urgent,0
hot,0


In [78]:
attacks_types = {
    'normal': 'normal',
    'back': 'dos',
    'buffer_overflow': 'u2r',
    'ftp_write': 'r2l',
    'guess_passwd': 'r2l',
    'imap': 'r2l',
    'ipsweep': 'probe',
    'land': 'dos',
    'loadmodule': 'u2r',
    'multihop': 'r2l',
    'neptune': 'dos',
    'nmap': 'probe',
    'perl': 'u2r',
    'phf': 'r2l',
    'pod': 'dos',
    'portsweep': 'probe',
    'rootkit': 'u2r',
    'satan': 'probe',
    'smurf': 'dos',
    'spy': 'r2l',
    'teardrop': 'dos',
    'warezclient': 'r2l',
    'warezmaster': 'r2l',
}

In [98]:
# kdd_df['attack_type'] = kdd_df['target'].apply(lambda x: attacks_types[x[:-1]])

In [57]:
kdd_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target,attack_type
0,0,tcp,http,SF,239,486,0,0,0,0,...,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.,normal
1,0,tcp,http,SF,235,1337,0,0,0,0,...,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.,normal
2,0,tcp,http,SF,219,1337,0,0,0,0,...,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.,normal
3,0,tcp,http,SF,217,2032,0,0,0,0,...,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.,normal


In [124]:
def format_data(kdd_df):
    new_cols = [col.strip() for col in cols.split(',')]
    new_cols.append('target')
    kdd_df.columns = new_cols
    kdd_df['attack_type'] = kdd_df['target'].apply(lambda x: attacks_types[x[:-1]])

In [125]:
format_data(kdd_df)

In [111]:
#Finding categorical features
num_cols = kdd_df._get_numeric_data().columns

cate_cols = list(set(kdd_df.columns)-set(num_cols))
cate_cols.remove('target')
cate_cols.remove('attack_type')

cate_cols

['service', 'protocol_type', 'flag']

remove redundant data (high correlation) to simplify the model without losing significant information.

high correlation (in this context) means that two columns are very similar in terms of their data values.

In [112]:
high_corr_threshold = 0.97

filtered_cols = [col for col in num_cols if col not in ['service', 'protocol_type', 'flag']]

corr_matrix = kdd_df[filtered_cols].corr()
high_corr_pairs = []

for col in corr_matrix.columns:
    related_cols = corr_matrix[col][abs(corr_matrix[col]) > high_corr_threshold].index.tolist()
    if len(related_cols) > 1:
        high_corr_pairs.append((col, related_cols))

keep_columns = set()
drop_columns = set()

grouped_columns = {}
for col, related_cols in high_corr_pairs:
    group = set([col] + related_cols)
    group_key = frozenset(group) # Avoid duplicates by using a frozenset
    if group_key not in grouped_columns:
        grouped_columns[group_key] = list(group)

for group_key, columns in grouped_columns.items():
    columns.sort()
    representative = columns[0]  # Choose the first column as representative
    keep_columns.add(representative)
    for col in columns[1:]:
        drop_columns.add(col)

group_number = 1
for group_key, columns in grouped_columns.items():
    representative = columns[0]
    print(f"Group {group_number} ({representative}-related):")
    print(f"Columns: {', '.join(columns)}")
    print(f"Drop: {', '.join(columns[1:])}")
    print(f"Keep: {representative} (as a representative)\n")
    group_number += 1


Group 1 (num_compromised-related):
Columns: num_compromised, num_root
Drop: num_root
Keep: num_compromised (as a representative)

Group 2 (dst_host_serror_rate-related):
Columns: dst_host_serror_rate, dst_host_srv_serror_rate, serror_rate, srv_serror_rate
Drop: dst_host_srv_serror_rate, serror_rate, srv_serror_rate
Keep: dst_host_serror_rate (as a representative)

Group 3 (dst_host_rerror_rate-related):
Columns: dst_host_rerror_rate, dst_host_srv_rerror_rate, rerror_rate, srv_rerror_rate
Drop: dst_host_srv_rerror_rate, rerror_rate, srv_rerror_rate
Keep: dst_host_rerror_rate (as a representative)

Group 4 (dst_host_same_srv_rate-related):
Columns: dst_host_same_srv_rate, dst_host_srv_count
Drop: dst_host_srv_count
Keep: dst_host_same_srv_rate (as a representative)



In [113]:
kdd_df.drop(['num_root'],axis = 1,inplace = True)
kdd_df.drop(['dst_host_srv_serror_rate', 'serror_rate', 'srv_serror_rate'],axis = 1,inplace = True)
kdd_df.drop(['dst_host_srv_rerror_rate', 'rerror_rate', 'srv_rerror_rate'],axis = 1,inplace = True)
kdd_df.drop(['dst_host_srv_count'],axis = 1,inplace = True)

# kdd_df.drop('num_root',axis = 1,inplace = True)
# kdd_df.drop('srv_serror_rate',axis = 1,inplace = True)
# kdd_df.drop('srv_rerror_rate',axis = 1, inplace=True)
# kdd_df.drop('dst_host_srv_serror_rate',axis = 1, inplace=True)
# kdd_df.drop('dst_host_serror_rate',axis = 1, inplace=True)
# kdd_df.drop('dst_host_rerror_rate',axis = 1, inplace=True)
# kdd_df.drop('dst_host_srv_rerror_rate',axis = 1, inplace=True)
# kdd_df.drop('dst_host_same_srv_rate',axis = 1, inplace=True)

In [103]:
kdd_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,srv_diff_host_rate,dst_host_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_rerror_rate,target,attack_type
0,0,tcp,http,SF,239,486,0,0,0,0,...,0.0,19,1.0,0.0,0.05,0.0,0.0,0.0,normal.,normal
1,0,tcp,http,SF,235,1337,0,0,0,0,...,0.0,29,1.0,0.0,0.03,0.0,0.0,0.0,normal.,normal
2,0,tcp,http,SF,219,1337,0,0,0,0,...,0.0,39,1.0,0.0,0.03,0.0,0.0,0.0,normal.,normal
3,0,tcp,http,SF,217,2032,0,0,0,0,...,0.0,49,1.0,0.0,0.02,0.0,0.0,0.0,normal.,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,0.0,59,1.0,0.0,0.02,0.0,0.0,0.0,normal.,normal


In [126]:
# protocol_type feature mapping
pmap = {'icmp': 0, 'tcp': 1, 'udp': 2}
kdd_df['protocol_type'] = kdd_df['protocol_type'].map(pmap)

In [127]:
# flag mapping
fmap = {'SF': 0, 'S0': 1, 'REJ':2, 'RSTR': 3, 'RSTO': 4, 'SH': 5 ,'S1': 6 , 'S2': 7, 'RSTOS0': 8, 'S3': 9 , 'OTH': 10}
kdd_df['flag'] = kdd_df['flag'].map(fmap)

In [128]:
kdd_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target,attack_type
0,0,1,http,0,239,486,0,0,0,0,...,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.,normal
1,0,1,http,0,235,1337,0,0,0,0,...,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.,normal
2,0,1,http,0,219,1337,0,0,0,0,...,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.,normal
3,0,1,http,0,217,2032,0,0,0,0,...,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.,normal
4,0,1,http,0,217,2032,0,0,0,0,...,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.,normal


In [129]:
kdd_df.drop('target',axis = 1,inplace = True)

# Scaling

In [130]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score